
# NCars Event-GNN Training (SAGE/GCN) — with AEGNN Async Eval

- Loads NCars sequences from `{train, validation, test}`
- Builds k-NN graphs in (x, y, t)
- Trains a **SAGE** or **GCN** encoder + classifier head
- Optionally wraps the **encoder only** with `AEGNNAsyncWrapper` for asynchronous evaluation
- Lets us limit the number of sequences per split for quick tests




In [18]:

# data folder containing 'training/', 'validation/', 'test/'
DATA = r"C:\Users\hanne\Documents\Hannes\Uni\Maastricht\Project\GNNBenchmark\src\Models\AEGNN\data\ncars"

# If events file has a specific name (else the dataset will auto-pick the first *.txt != is_car.txt)
EVENTS_NAME = None  # should be "events.txt"

# Sensor width,height (string "W,H") or None to normalize per-sequence
SENSOR_WH = "304,240"  # or None

# Graph building
K = 8
MAX_EVENTS = 1000  # reduce (e.g., 1000) for faster first run

# Model
MODEL = "sage"     # "sage" or "gcn"
HIDDEN = 64
DROPOUT = 0.2

# Training
EPOCHS = 15
BATCH_SIZE = 32
LR = 1e-3
WD = 0.0

# Use only first N (20) sequences per split (0 = all), for quick testing
LIMIT_PER_SPLIT = 100

# Async encoder wrapping at evaluation
ASYNC_EVAL = True


In [19]:

import sys, os, pathlib
project_root_candidates = [
    pathlib.Path.cwd(),                                  # current directory
    pathlib.Path.cwd().parents[0],                       # 1 level up
    pathlib.Path.cwd().parents[1],                       # 2 levels up
]
for pr in project_root_candidates:
    if str(pr) not in sys.path:
        sys.path.append(str(pr))

from pathlib import Path
import torch
import torch.nn as nn
from torch.optim import Adam
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

# moduls needed
from ProcessNCars import NCarsEventsGraphDataset
from GCNEncoder import GCNEncoder
from SAGEEncoder import SAGEEncoder
from FullModel_GCN_SAGE import FullModel, ClassifierHead
from train_ncars_from_events import train_one_epoch, evaluate, take_first
from ProcessNCars1 import NCarsEventsGraphDataset1

#

# Data Loading


In [20]:
# dataset setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sensor_wh = None
if SENSOR_WH:
    w, h = [int(s.strip()) for s in SENSOR_WH.split(",")]
    sensor_wh = (w, h)

root = Path(DATA)
train_dir = root / "training"#change to your data path
val_dir   = root / "validation"
test_dir  = root / "test"
for d in [train_dir, val_dir, test_dir]:
    assert d.exists(), f"Missing split directory: {d}"

precompute_gcn = (MODEL == "gcn")

train_ds = NCarsEventsGraphDataset1(str(train_dir), EVENTS_NAME, K, MAX_EVENTS,
                                   sensor_wh, precompute_gcn)
val_ds   = NCarsEventsGraphDataset1(str(val_dir),   EVENTS_NAME, K, MAX_EVENTS,
                                   sensor_wh, precompute_gcn)
test_ds  = NCarsEventsGraphDataset1(str(test_dir),  EVENTS_NAME, K, MAX_EVENTS,
                                   sensor_wh, precompute_gcn)

# Limit for quick runs and testing
if LIMIT_PER_SPLIT and LIMIT_PER_SPLIT > 0:
    train_ds = take_first(train_ds, LIMIT_PER_SPLIT)
    val_ds   = take_first(val_ds, LIMIT_PER_SPLIT)
    # Uncomment to also limit test set:
    test_ds  = take_first(test_ds, LIMIT_PER_SPLIT)

print("Datasets ready.",
      "\n  train:", len(train_ds),
      "\n  val:  ", len(val_ds),
      "\n  test: ", len(test_ds))


Datasets ready. 
  train: 100 
  val:   100 
  test:  100


# Model, Loaders, Optimizer


In [21]:
# model
sample: Data = train_ds[0]
in_ch = sample.x.size(-1)
num_classes = int(max(sample.y.max().item(), 1) + 1)

if MODEL == "sage":
    encoder = SAGEEncoder(in_ch, hid=HIDDEN)
else:
    encoder = GCNEncoder(in_ch, hid=HIDDEN)

head = ClassifierHead(hid=HIDDEN, num_classes=num_classes, dropout=DROPOUT, bias=True)
model = FullModel(encoder, head).to(device)

# loaders
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False)

# Optimizer & loss
opt = Adam(model.parameters(), lr=LR, weight_decay=WD)
crit = nn.CrossEntropyLoss()

print(model)


FullModel(
  (encoder): SAGEEncoder(
    (c1): SAGEConv(1, 64, aggr=mean)
    (c2): SAGEConv(64, 64, aggr=mean)
  )
  (head): ClassifierHead(
    (fc): Linear(in_features=64, out_features=2, bias=True)
  )
)


# Training Loop

In [22]:

#training loop
best_val, best_state = 0.0, None
for epoch in range(1, EPOCHS + 1):
    tr_loss, tr_acc = train_one_epoch(model, train_loader, device, opt, crit)
    va_loss, va_acc = evaluate(model, val_loader, device, crit)
    if va_acc > best_val:
        best_val = va_acc
        best_state = {k: v.detach().cpu() for k, v in model.state_dict().items()}
    print(f"Epoch {epoch:03d} | train {tr_loss:.4f}/{tr_acc:.3f} | val {va_loss:.4f}/{va_acc:.3f}")

if best_state:
    model.load_state_dict(best_state)


Epoch 001 | train 0.7003/0.490 | val 0.6937/0.490
Epoch 002 | train 0.6890/0.560 | val 0.6918/0.560
Epoch 003 | train 0.6916/0.570 | val 0.6922/0.560
Epoch 004 | train 0.6919/0.500 | val 0.6940/0.480
Epoch 005 | train 0.6948/0.520 | val 0.6938/0.480
Epoch 006 | train 0.6911/0.560 | val 0.6936/0.480
Epoch 007 | train 0.6954/0.530 | val 0.6919/0.510
Epoch 008 | train 0.6857/0.560 | val 0.6908/0.570
Epoch 009 | train 0.6902/0.550 | val 0.6895/0.570
Epoch 010 | train 0.6892/0.530 | val 0.6889/0.560
Epoch 011 | train 0.6971/0.550 | val 0.6888/0.540
Epoch 012 | train 0.6899/0.580 | val 0.6901/0.570
Epoch 013 | train 0.6897/0.590 | val 0.6910/0.550
Epoch 014 | train 0.6917/0.540 | val 0.6921/0.520
Epoch 015 | train 0.6866/0.560 | val 0.6918/0.520


# Asynchronous Evaluation

In [23]:


# async for evaluation, only part taken from AEGNN
if ASYNC_EVAL:
    try:
        from AEGNNwrapper import AEGNNAsyncWrapper
        enc_async = AEGNNAsyncWrapper(model.encoder)
        print("[AEGNN] is_async=", getattr(enc_async, "is_async", False),
              "| why_not=", getattr(enc_async, "why_not_async", None))
        model.encoder = enc_async.to(device)
    except Exception as e:
        print("[AEGNN] Wrapper import failed; continuing without async. Reason:", repr(e))


[AEGNN] is_async= True | why_not= None


In [24]:
from tqdm import tqdm  # pip install tqdm (optional); else replace with a simple loop
bad = []
for i in range(len(test_ds)):
    try:
        _ = test_ds[i]
    except Exception as e:
        bad.append((i, str(test_ds.seq_dirs[i]), repr(e)))
        print("BAD:", i, test_ds.seq_dirs[i], e)

print("bad count:", len(bad))
if bad:
    print("examples:", bad[:5])


bad count: 0


# Testing


In [26]:
# test
te_loss, te_acc = evaluate(model, test_loader, device, crit)
print(f"Test  | loss {te_loss:.4f} acc {te_acc:.3f}")

AttributeError: 'SAGEConv' object has no attribute 'weight'